In [ ]:
from google.colab import files
uploaded= files.upload()

Saving IMDB Dataset.csv to IMDB Dataset.csv


In [ ]:
import pandas as pd
df=pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(50000, 2)

In [ ]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,negative
freq,5,25000


In [ ]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
df['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
import spacy
import re,string, unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from bs4 import BeautifulSoup

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#tokenzation of texts
tokenizer = ToktokTokenizer()
#setting english stopwords
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
#removing the noisy text
def noiseremoval_text(text):
  soup = BeautifulSoup(text, "html.parser")
  text = soup.get_text()
  text = re.sub('\[[^]]*\]', '', text)
  return text

In [ ]:
#apply function on review column
df['review'] = df['review'].apply(noiseremoval_text)

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# stemming

In [ ]:
#stemming the text
def stemmer(text):
  ps =  nltk.porter.PorterStemmer()
  text = ' '.join([ps.stem(word) for word in text.split()])
  return text

In [ ]:
df['review'] = df['review'].apply(stemmer)

In [ ]:
df.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,A wonder littl production. the film techniqu i...,positive
2,I thought thi wa a wonder way to spend time on...,positive
3,basic there' a famili where a littl boy (jake)...,negative
4,"petter mattei' ""love in the time of money"" is ...",positive


removing the stopwords

In [ ]:
def removing_stopwords(text, is_lower_case=False):
  tokenizers = ToktokTokenizer()
  tokens = tokenizers.tokenize(text)
  tokens = [i.strip() for i in tokens]
  if is_lower_case:
    filtokens = [i for i in tokens if token not in stopwords]
  else:
    filtokens = [i for i in tokens if i.lower() not in stopwords]
  fltered_texts = ' '.join(filtokens)
  return fltered_texts

In [ ]:
df['review'] = df['review'].apply(removing_stopwords)

In [ ]:
df.head()

,review,sentiment
0,one review ha mention watch 1 Oz episod ' hook...,positive
1,wonder littl production. film techniqu veri un...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic ' famili littl boy ( jake ) think ' zomb...,negative
4,"petter mattei ' "" love time money "" visual stu...",positive


In [ ]:
train_reviews = df.review[:30000]
test_reviews = df.review[30000:]

bag of words

In [ ]:
#count vectorizer for bag of words
cv = CountVectorizer(min_df=0, max_df=1, binary = False, ngram_range=(1, 3))
#transformed train reviews
cv_train = cv.fit_transform(train_reviews)
#transform test reviews
cv_test = cv.transform(test_reviews)

print('bag of words of cv_train :', cv_train.shape)
print('bag of words of cv_test :', cv_test.shape)

bag of words of cv_train : (30000, 4954557)
bag of words of cv_test : (20000, 4954557)


TF-IDF

In [ ]:
tf = TfidfVectorizer(min_df=0, max_df=1, use_idf=True, ngram_range=(1,3))
tf_train = tf.fit_transform(train_reviews)
tf_test = tf.transform(test_reviews)
print('tfidf_train :', tf_train.shape)
print('tfidf_test :', tf_test.shape)

tfidf_train : (30000, 4954557)
tfidf_test : (20000, 4954557)


label encoding

In [ ]:
#labeling the sentiment data
label = LabelBinarizer()
sentiment_data = label.fit_transform(df['sentiment'])
print(sentiment_data.shape)

(50000, 1)


In [ ]:
sentiment_data

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
train_data = df.sentiment[:30000]
test_data = df.sentiment[30000:]

In [ ]:
#training the model
logreg = LogisticRegression(penalty='l2', max_iter=500, C=1, random_state=42)
#fitting the model for bag of words
lr_bow = logreg.fit(cv_train, train_data)
print(lr_bow)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
#predicting the model for bag of words
lr_bow_predict = logreg.predict(cv_test)
print(lr_bow_predict)

['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']


In [ ]:
lr_bow_score = accuracy_score(test_data, lr_bow_predict)
lr_bow_score

0.59215

In [ ]:
#fitiing the model for tfidf features
lr_tfidf = logreg.fit(tf_train, train_data)
print(lr_tfidf)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
#predicting the moel for tfidf features
lr_tfidf_predict = logreg.predict(tf_test)
print(lr_tfidf_predict)

['negative' 'negative' 'negative' ... 'negative' 'positive' 'positive']


In [ ]:
lr_tfidf_score = accuracy_score(test_data, lr_tfidf_predict)
lr_tfidf_score

0.7426